In [1]:
import pandas as pd
import os
import sklearn
import numpy as np
from utils import split_dataset

Load data

In [2]:
covariates = pd.read_csv("./data/bigdata.csv", index_col="Dates", skiprows=5)
covariates.index = pd.to_datetime(covariates.index)

labels = pd.read_csv("./data/jkse.csv", skiprows=5, index_col="Dates")
labels.index = pd.to_datetime(labels.index)

labels = labels.rename(columns={"PX_LAST": "JKSE_PRICE"})
labels['PCT_CHANGE_20_JKSE'] = ((labels['JKSE_PRICE'].shift(-20) - labels['JKSE_PRICE']) / labels['JKSE_PRICE']) * 100

Split the dataset per ticker

In [3]:
covlist = split_dataset(covariates)

## Create technical covariates


### PE Band & P/E Ratio  
$$
 \text{P/E Ratio} = \frac{1}{\text{EARN\_YLD}} 
$$

* $PE Band_q $ is the q-th percentile of P/E Ratio in a 60 trading days rolling window

### Mean Average Convergence/Divergence (MACD)

- MACD is the difference of short term $\text{EMA}_{12}$ and long term $\text{EMA}_{26}$ (signal for momentum)
- MACD Signal is the signal line of 9 day EMA of the MACD
- MACD Histogram is the difference between the MACD line and the signal line

In [4]:
colnames = covlist[0].columns.tolist()
for cov in covlist:
    # unify column names
    cov.columns = colnames
for cov in covlist:
    # Volume = Turnover / Close Price
    cov['VOLUME'] = cov['TURNOVER'] / cov['PX_LAST']
    # Calculate percent change * 100
    cov['PCT_CHANGE_20'] = ((cov['PX_LAST'].shift(-20) - cov['PX_LAST']) / cov['PX_LAST']) * 100
    # Ratio 10/30 = mean volume ratio for the last 10 days / mean volume ratio for the last 30 days
    cov['VOL_RATIO_10_20'] = cov['VOLUME'].rolling(window=10).mean() / cov['VOLUME'].rolling(window=20).mean()
    cov['VOL_RATIO_20_40'] = cov['VOLUME'].rolling(window=20).mean() / cov['VOLUME'].rolling(window=40).mean()
    cov['VOL_RATIO_40_80'] = cov['VOLUME'].rolling(window=40).mean() / cov['VOLUME'].rolling(window=80).mean()
    cov['VOL_RATIO_80_120'] = cov['VOLUME'].rolling(window=80).mean() / cov['VOLUME'].rolling(window=120).mean()
    
    # PE Band
    cov['PE_Ratio'] = 1 / cov['EARN_YLD']
    win = 60  # Set the rolling window period
    cov['PE_Band_25'] = cov['PE_Ratio'].rolling(win).quantile(0.25)
    cov['PE_Band_50'] = cov['PE_Ratio'].rolling(win).quantile(0.50)
    cov['PE_Band_75'] = cov['PE_Ratio'].rolling(win).quantile(0.75)

    # Calculate the 12-day EMA of PX_LAST
    ema_12 = cov['PX_LAST'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-day EMA of PX_LAST
    ema_26 = cov['PX_LAST'].ewm(span=26, adjust=False).mean()

    # Calculate MACD
    cov['MACD'] = ema_12 - ema_26

    # Calculate the Signal line (9-day EMA of MACD)
    cov['MACD_Signal'] = cov['MACD'].ewm(span=9, adjust=False).mean()

    # Optionally, you can also calculate the MACD Histogram (the difference between MACD and Signal line)
    cov['MACD_Histogram'] = cov['MACD'] - cov['MACD_Signal']
    
    # Example: Momentum Indicator for various lags
    lags = [10, 20, 30, 60, 120]
    for lag in lags:
        cov[f'MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'TURNOVER_{lag}'] = cov['TURNOVER'].rolling(window=lag).mean()
        cov[f'PX_MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'PX_REVERSAL_{lag}'] = cov['PX_LAST'].shift(lag) / cov['PX_LAST']
        cov[f'VOLATILITY_{lag}'] = cov['PX_LAST'].rolling(window=lag).std()
        cov[f'VOLUME_STD_{lag}'] = cov['VOLUME'].rolling(window=lag).std()

In [5]:
# Remove duplicate indices if any
labels = labels[~labels.index.duplicated(keep='first')]

for i in range(len(covlist)):
    cov = covlist[i]
    cov = cov[~cov.index.duplicated(keep='first')]

    # Explicitly create a copy of the cov DataFrame to avoid SettingWithCopyWarning
    cov_copy = cov.copy()

    # Align the DataFrames on their indices (dates)
    aligned_df = labels.join(cov_copy[['PCT_CHANGE_20']], how='inner')

    # Calculate the difference and store it in cov_copy DataFrame
    cov_copy.loc[aligned_df.index, 'DELTA_20_CHANGE'] = aligned_df['PCT_CHANGE_20'] - aligned_df['PCT_CHANGE_20_JKSE']

    # Update the original DataFrame in covlist
    covlist[i] = cov_copy


In [6]:
tickernames = [col[:4] for col in pd.read_csv("./data/bigdata.csv", skiprows=3).columns if not col.startswith("Unnamed")]

/var/folders/5w/sk_z507j78j69yh904rh10440000gn/T/ipykernel_3220/2531172465.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,

In [7]:
for i in range(len(covlist)):
    covlist[i]['Ticker'] = tickernames[i]

In [8]:
def unique_index(df, suffix):
    df_copy = df.copy()
    new_index = [f"{date.strftime('%Y-%m-%d')}-{suffix}" for date in df.index]
    df_copy.index = new_index
    return df_copy

Create reindexed covlist + pool to find quantiles + insert index back to other covariates

In [9]:
covlist_reindex = []
for idx, cov in enumerate(covlist):
    cov['Ticker'] = cov['Ticker'].unique()[0]
    covlist_reindex.append(unique_index(cov, idx))
    
pooled_df = pd.DataFrame()
for i, df in enumerate(covlist_reindex):
    pooled_df = pd.concat([pooled_df, df['DELTA_20_CHANGE']])

pooled_df['DELTA_20_QUINTILES'] = pd.qcut(pooled_df[0], q=5, labels=range(1, 6))

for i, df in enumerate(covlist_reindex):
    df['DELTA_20_QUINTILES'] = pooled_df.loc[df.index, 'DELTA_20_QUINTILES']
    covlist_reindex[i] = df

In [10]:
for cov in covlist_reindex:
    cov['TOP_5'] = cov['DELTA_20_QUINTILES'].apply(lambda x: 1 if x == 5 else 0)
    cov['TOP_5'] = cov['TOP_5'].fillna(0).astype(int)  # Fill NaN values with 0 before conversion

Drop NaNs

In [11]:
for i in range(len(covlist_reindex)):
    covlist_reindex[i] = covlist_reindex[i].dropna()
    covlist_reindex[i] = covlist_reindex[i].reset_index(drop=False)

save data

In [22]:
def rolling_window_save(train_window=252, test_window=60):
    tdf = []
    vdf = []

    for cov in covlist_reindex:
        # Remove specific columns and reset the index but keep the original one
        cov = cov.drop(['DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1)

        def split_windows(df):
            # Create train and test windows based on the provided lengths
            for start in range(0, len(df) - train_window - test_window, train_window + test_window):
                end_train = start + train_window
                end_test = end_train + test_window

                train_df = df.iloc[start:end_train].reset_index(drop=False)
                test_df = df.iloc[end_train:end_test].reset_index(drop=False)

                tdf.append(train_df)
                vdf.append(test_df)

        # Apply the split function over the entire DataFrame
        cov.groupby((cov.index // (train_window + test_window)).astype(int)).apply(split_windows)

    return tdf, vdf, cov

tdf, vdf, cov = rolling_window_save(252, 60)

In [23]:
cov

,index,EARN_YLD,PX_TO_BOOK_RATIO,PX_TO_SALES_RATIO,RETURN_ON_ASSET,RETURN_COM_EQY,CUR_MKT_CAP,TRAIL_12M_NET_INC_GROWTH,OPER_INC_GROWTH,VWAP_STANDARD_DEV,...,VOLUME_STD_60,MOMENTUM_120,TURNOVER_120,PX_MOMENTUM_120,PX_REVERSAL_120,VOLATILITY_120,VOLUME_STD_120,Ticker,DELTA_20_QUINTILES,TOP_5
0,2010-01-29-26,6.8856,6.9511,4.4291,38.4583,56.2448,39631067.58,59.7248,-19.1733,14.0,...,8.449401e+06,1.278810,6.598283e+10,1.278810,0.781977,343.697140,1.565632e+07,PTBA,1,0
1,2010-02-01-26,7.0917,6.7491,4.3004,38.4583,56.2448,38479001.66,59.7248,-19.1733,22.0,...,8.471478e+06,1.241636,6.596687e+10,1.241636,0.805389,342.445318,1.565609e+07,PTBA,2,0
2,2010-02-02-26,7.1777,6.6683,4.2489,38.4583,56.2448,38018175.29,59.7248,-19.1733,29.0,...,8.467346e+06,1.245283,6.452191e+10,1.245283,0.803030,340.461038,1.458413e+07,PTBA,2,0
3,2010-02-03-26,7.0078,6.8299,4.3519,38.4583,56.2448,38939828.03,59.7248,-19.1733,13.0,...,8.395199e+06,1.310078,6.442713e+10,1.310078,0.763314,337.982138,1.459335e+07,PTBA,2,0
4,2010-02-04-26,7.0286,6.8097,4.3390,38.4583,56.2448,38824621.44,59.7248,-19.1733,24.0,...,8.404193e+06,1.301158,6.400069e+10,1.301158,0.768546,335.418955,1.457350e+07,PTBA,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3510,2024-07-11-26,18.3049,1.5135,0.7463,12.6668,28.4573,29377681.09,-41.6189,-14.0126,5.0,...,2.035380e+07,0.992218,5.156913e+10,0.992218,1.007843,209.227454,1.730409e+07,PTBA,4,0
3511,2024-07-12-26,18.4496,1.5016,0.7405,12.6668,28.4573,29147267.90,-41.6189,-14.0126,7.0,...,2.040272e+07,0.984436,5.162393e+10,0.984436,1.015810,209.505871,1.728872e+07,PTBA,4,0
3512,2024-07-15-26,18.5228,1.4957,0.7375,12.6668,28.4573,29032061.31,-41.6189,-14.0126,14.0,...,2.042210e+07,0.980545,5.176246e+10,0.980545,1.019841,209.862633,1.725773e+07,PTBA,4,0
3513,2024-07-16-26,18.4496,1.5016,0.7405,12.6668,28.4573,29147267.90,-41.6189,-14.0126,6.0,...,2.043620e+07,0.976834,5.172539e+10,0.976834,1.023715,210.252882,1.726585e+07,PTBA,4,0


In [ ]:
covlist_reindex[3]['Ticker'].head()

In [ ]:
all_vdf[10]['Ticker']

In [ ]:
valid = pd.concat(all_vdf, axis=0)
valid.index = valid['index']
valid.drop('index', axis=1, inplace=True)

In [ ]:
valid.loc[:, ['PX_LAST', "VOLATILITY_20", 'Ticker', 'DELTA_20_CHANGE', "DELTA_20_QUINTILES", "TOP_5", "PCT_CHANGE_20"]]

In [ ]:
folder_name = 'covset2'
os.makedirs(folder_name, exist_ok=True)
for i in range(len(tdf)):
    # tdf[i].to_csv(f'{folder_name}/train_{i}.csv', index=False)
    vdf[i].to_csv(f'{folder_name}/valid_{i}_pred.csv', index=False)

In [ ]:
valid.to_csv(f'{folder_name}/valid_pred_all.csv', index=True)